# 2110433 - Computer Vision (2023/2)
## Lab 11 - Dynamic Vision & 3D
In this lab, we will learn to describe image motion from images. This notebook includes both coding and written questions. Please hand in this notebook file with all outputs and your answer

!pip install opencv-contrib-python

In [ ]:
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

## 1. Background Subtraction
This tutorial was originally from [here](https://docs.opencv.org/3.4/d1/dc5/tutorial_background_subtraction.html)

- Background subtraction (BS) is a common and widely used technique for generating a foreground mask (namely, a binary image containing the pixels belonging to moving objects in the scene) by using static cameras.
- As the name suggests, BS calculates the foreground mask performing a subtraction between the current frame and a background model, containing the static part of the scene or, more in general, everything that can be considered as background given the characteristics of the observed scene.



<img src="https://docs.opencv.org/3.4/Background_Subtraction_Tutorial_Scheme.png" />

A [cv2.VideoCapture](https://docs.opencv.org/3.4/d8/dfe/classcv_1_1VideoCapture.html) object is used to read the input video or input images sequence.

In [ ]:
capture = cv2.VideoCapture('assets/vtest.avi')

A [cv2.BackgroundSubtractor](https://docs.opencv.org/3.4/d7/df6/classcv_1_1BackgroundSubtractor.html) object will be used to generate the foreground mask. In this example, default parameters are used, but it is also possible to declare specific parameters in the create function.

We use MOG background subtractor for this example.

It uses a method to model each background pixel by a mixture of K Gaussian distributions (K = 3 to 5). The weights of the mixture represent the time proportions that those colours stay in the scene. The probable background colours are the ones which stay longer and more static.

<img src="https://miro.medium.com/max/1400/0*1EkPhFtPTRgmHumI.png" />

In [ ]:
backSub = cv2.bgsegm.createBackgroundSubtractorMOG()

Every frame is used both for calculating the foreground mask and for updating the background. If you want to change the learning rate used for updating the background model, it is possible to set a specific learning rate by passing a parameter to the apply method.

You can read more about "apply" function by clicking [here](https://docs.opencv.org/3.4/d7/df6/classcv_1_1BackgroundSubtractor.html#aa735e76f7069b3fa9c3f32395f9ccd21)

In [ ]:
ret, frame = capture.read()
fgMask = backSub.apply(frame)
plt.imshow(fgMask)
plt.show()

Show the result!

In [ ]:
capture = cv2.VideoCapture('assets/vtest.avi')
backSub = cv2.bgsegm.createBackgroundSubtractorMOG()
while True:
    ret, frame = capture.read()
    if frame is None:
        break
    
    fgMask = backSub.apply(frame)
    
    
    cv2.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv2.putText(frame, str(capture.get(cv2.CAP_PROP_POS_FRAMES)), (15, 15),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    
    cv2.imshow('Frame', frame)
    cv2.imshow('KNN Mask', fgMask)
    
    keyboard = cv2.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

There are other background subtraction algorithms as shown in the image below.

<img src="https://docs.opencv.org/3.4/d7/df6/classcv_1_1BackgroundSubtractor.png"></img>

In the next block, we compare 4 background subtraction algorithms.

In [ ]:
capture = cv2.VideoCapture('assets/vtest.avi')


backSubKNN = cv2.createBackgroundSubtractorKNN()
backSubMOG2 = cv2.createBackgroundSubtractorMOG2()
backSubMOG = cv2.bgsegm.createBackgroundSubtractorMOG()
backSubGMG = cv2.bgsegm.createBackgroundSubtractorGMG()


while True:
    ret, frame = capture.read()
    if frame is None:
        break
    
    frame = cv2.resize(frame, (400, 300)) 
    
    fgMaskKNN = backSubKNN.apply(frame)
    fgMaskMOG = backSubMOG.apply(frame)
    fgMaskMOG2 = backSubMOG2.apply(frame)
    fgMaskGMG = backSubGMG.apply(frame)
    
    
    cv2.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv2.putText(frame, str(capture.get(cv2.CAP_PROP_POS_FRAMES)), (15, 15),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    
    original =  cv2.imshow('Frame', frame)
    
    
    cv2.imshow('KNN Mask', fgMaskKNN)
    cv2.imshow('MOG Mask', fgMaskMOG)
    cv2.imshow('MOG2 Mask', fgMaskMOG2)
    cv2.imshow('GMG Mask', fgMaskGMG)
    
    keyboard = cv2.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

## 2. Optical Flow in OpenCV
This tutorial was originally from [here](https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html)

Optical flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movement of object or camera. It is 2D vector field where each vector is a displacement vector showing the movement of points from first frame to second. Consider the image below (Image Courtesy: [Wikipedia article on Optical Flow](https://en.wikipedia.org/wiki/Optical_flow)).

<img src='https://docs.opencv.org/3.4/optical_flow_basic1.jpg'/>
image
It shows a ball moving in 5 consecutive frames. The arrow shows its displacement vector. 



Optical flow works on several assumptions:
- The pixel intensities of an object do not change between consecutive frames.
- Neighbouring pixels have similar motion.

### Lucas-Kanade Optical Flow 

We have seen an assumption before, that all the neighbouring pixels will have similar motion. Lucas-Kanade method takes a 3x3 patch around the point. So all the 9 points have the same motion.

So from the user point of view, the idea is simple, we give some points to track, we receive the optical flow vectors of those points. But again there are some problems. Until now, we were dealing with small motions, so it fails when there is a large motion. To deal with this we use pyramids. When we go up in the pyramid, small motions are removed and large motions become small motions. So by applying Lucas-Kanade there, we get optical flow along with the scale.


You can see the video below for more detail.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo(id="yFX_N5p0kO0")

To decide the points to detect optical flow, we use [cv2.goodFeaturesToTrack()](https://docs.opencv.org/3.4/dd/d1a/group__imgproc__feature.html#ga1d6bb77486c8f92d79c8793ad995d541). We take the first frame, detect some Shi-Tomasi corner points in it

In [ ]:
cap = cv2.VideoCapture('assets/slow_traffic_small.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

Then, we iteratively track those points using Lucas-Kanade optical flow. For the function [cv2.calcOpticalFlowPyrLK()](https://docs.opencv.org/3.4/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323) we pass the previous frame, previous points and next frame. It returns next points along with some status numbers which has a value of 1 if next point is found, else zero. We iteratively pass these next points as previous points in next step.

In [ ]:
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv2.add(frame, mask)
    cv2.imshow('frame', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
cv2.destroyAllWindows()

#### Dense Optical Flow

Lucas-Kanade method computes optical flow for a sparse feature set (in our example, corners detected using Shi-Tomasi algorithm). OpenCV provides another algorithm to find the dense optical flow. It computes the optical flow for all the points in the frame. It is based on Gunnar Farneback's algorithm which is explained in "Two-Frame Motion Estimation Based on Polynomial Expansion" by Gunnar Farneback in 2003.

Below sample shows how to find the dense optical flow using above algorithm. We get a 2-channel array with optical flow vectors, (u,v). We find their magnitude and direction. We color code the result for better visualization. Direction corresponds to Hue value of the image. Magnitude corresponds to Value plane.

In [ ]:

cap = cv2.VideoCapture("assets/slow_traffic_small.mp4")
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    cv2.imshow('Frame', frame2)
    cv2.imshow('frame2', bgr)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    prvs = next
cv2.destroyAllWindows()

## 3. Camera Calibration
This tutorial was originally from [here](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html)

<div class="textblock"><h1><a class="anchor" id="autotoc_md1157"></a>
Goal</h1>
<p>In this section, we will learn about</p>
<ul>
<li>types of distortion caused by cameras</li>
<li>how to find the intrinsic and extrinsic properties of a camera</li>
<li>how to undistort images based off these properties</li>
</ul>
<h1><a class="anchor" id="autotoc_md1158"></a>
Basics</h1>
<p>Some pinhole cameras introduce significant distortion to images. Two major kinds of distortion are radial distortion and tangential distortion.</p>
<p>Radial distortion causes straight lines to appear curved. Radial distortion becomes larger the farther points are from the center of the image. For example, one image is shown below in which two edges of a chess board are marked with red lines. But, you can see that the border of the chess board is not a straight line and doesn't match with the red line. All the expected straight lines are bulged out. Visit <a href="https://en.wikipedia.org/wiki/Distortion_%28optics%29" target="_blank" class="keychainify-checked">Distortion (optics)</a> for more details.</p>
<div class="image">
<img src="https://docs.opencv.org/4.x/calib_radial.jpg" alt="">
<div class="caption">
image</div></div>
<p>Radial distortion can be represented as follows:</p>
<p class="formulaDsp">
<mjx-container class="MathJax CtxtMenu_Attached_0" jax="CHTML" display="true" role="presentation" tabindex="0" ctxtmenu_counter="0" style="font-size: 118.2%; position: relative;"><mjx-math display="true" class="MJX-TEX" aria-hidden="true"><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c78"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-texatom size="s" texclass="ORD"><mjx-mi class="mjx-i"><mjx-c class="mjx-c64"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c69"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c73"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c6F"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c65"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c64"></mjx-c></mjx-mi></mjx-texatom></mjx-script></mjx-msub><mjx-mo class="mjx-n" space="4"><mjx-c class="mjx-c3D"></mjx-c></mjx-mo><mjx-mi class="mjx-i" space="4"><mjx-c class="mjx-c78"></mjx-c></mjx-mi><mjx-mo class="mjx-n"><mjx-c class="mjx-c28"></mjx-c></mjx-mo><mjx-mn class="mjx-n"><mjx-c class="mjx-c31"></mjx-c></mjx-mn><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-msub space="3"><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c6B"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c31"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-msup><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-script style="vertical-align: 0.413em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msup><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-msub space="3"><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c6B"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-msup><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-script style="vertical-align: 0.413em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c34"></mjx-c></mjx-mn></mjx-script></mjx-msup><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-msub space="3"><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c6B"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c33"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-msup><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-script style="vertical-align: 0.413em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c36"></mjx-c></mjx-mn></mjx-script></mjx-msup><mjx-mo class="mjx-n"><mjx-c class="mjx-c29"></mjx-c></mjx-mo><mjx-mspace></mjx-mspace><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c79"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-texatom size="s" texclass="ORD"><mjx-mi class="mjx-i"><mjx-c class="mjx-c64"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c69"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c73"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c6F"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c65"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c64"></mjx-c></mjx-mi></mjx-texatom></mjx-script></mjx-msub><mjx-mo class="mjx-n" space="4"><mjx-c class="mjx-c3D"></mjx-c></mjx-mo><mjx-mi class="mjx-i" space="4"><mjx-c class="mjx-c79"></mjx-c></mjx-mi><mjx-mo class="mjx-n"><mjx-c class="mjx-c28"></mjx-c></mjx-mo><mjx-mn class="mjx-n"><mjx-c class="mjx-c31"></mjx-c></mjx-mn><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-msub space="3"><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c6B"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c31"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-msup><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-script style="vertical-align: 0.413em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msup><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-msub space="3"><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c6B"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-msup><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-script style="vertical-align: 0.413em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c34"></mjx-c></mjx-mn></mjx-script></mjx-msup><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-msub space="3"><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c6B"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c33"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-msup><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-script style="vertical-align: 0.413em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c36"></mjx-c></mjx-mn></mjx-script></mjx-msup><mjx-mo class="mjx-n"><mjx-c class="mjx-c29"></mjx-c></mjx-mo></mjx-math><mjx-assistive-mml role="presentation" unselectable="on" display="block"><math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><msub><mi>x</mi><mrow><mi>d</mi><mi>i</mi><mi>s</mi><mi>t</mi><mi>o</mi><mi>r</mi><mi>t</mi><mi>e</mi><mi>d</mi></mrow></msub><mo>=</mo><mi>x</mi><mo stretchy="false">(</mo><mn>1</mn><mo>+</mo><msub><mi>k</mi><mn>1</mn></msub><msup><mi>r</mi><mn>2</mn></msup><mo>+</mo><msub><mi>k</mi><mn>2</mn></msub><msup><mi>r</mi><mn>4</mn></msup><mo>+</mo><msub><mi>k</mi><mn>3</mn></msub><msup><mi>r</mi><mn>6</mn></msup><mo stretchy="false">)</mo><mspace linebreak="newline"></mspace><msub><mi>y</mi><mrow><mi>d</mi><mi>i</mi><mi>s</mi><mi>t</mi><mi>o</mi><mi>r</mi><mi>t</mi><mi>e</mi><mi>d</mi></mrow></msub><mo>=</mo><mi>y</mi><mo stretchy="false">(</mo><mn>1</mn><mo>+</mo><msub><mi>k</mi><mn>1</mn></msub><msup><mi>r</mi><mn>2</mn></msup><mo>+</mo><msub><mi>k</mi><mn>2</mn></msub><msup><mi>r</mi><mn>4</mn></msup><mo>+</mo><msub><mi>k</mi><mn>3</mn></msub><msup><mi>r</mi><mn>6</mn></msup><mo stretchy="false">)</mo></math></mjx-assistive-mml></mjx-container>
</p>
<p>Similarly, tangential distortion occurs because the image-taking lense is not aligned perfectly parallel to the imaging plane. So, some areas in the image may look nearer than expected. The amount of tangential distortion can be represented as below:</p>
<p class="formulaDsp">
<mjx-container class="MathJax CtxtMenu_Attached_0" jax="CHTML" display="true" role="presentation" tabindex="0" ctxtmenu_counter="1" style="font-size: 118.2%; position: relative;"><mjx-math display="true" class="MJX-TEX" aria-hidden="true"><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c78"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-texatom size="s" texclass="ORD"><mjx-mi class="mjx-i"><mjx-c class="mjx-c64"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c69"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c73"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c6F"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c65"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c64"></mjx-c></mjx-mi></mjx-texatom></mjx-script></mjx-msub><mjx-mo class="mjx-n" space="4"><mjx-c class="mjx-c3D"></mjx-c></mjx-mo><mjx-mi class="mjx-i" space="4"><mjx-c class="mjx-c78"></mjx-c></mjx-mi><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c5B"></mjx-c></mjx-mo><mjx-mn class="mjx-n"><mjx-c class="mjx-c32"></mjx-c></mjx-mn><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c70"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c31"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-mi class="mjx-i"><mjx-c class="mjx-c78"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c79"></mjx-c></mjx-mi><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-msub space="3"><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c70"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-mo class="mjx-n"><mjx-c class="mjx-c28"></mjx-c></mjx-mo><mjx-msup><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-script style="vertical-align: 0.413em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msup><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-mn class="mjx-n" space="3"><mjx-c class="mjx-c32"></mjx-c></mjx-mn><mjx-msup><mjx-mi class="mjx-i"><mjx-c class="mjx-c78"></mjx-c></mjx-mi><mjx-script style="vertical-align: 0.413em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msup><mjx-mo class="mjx-n"><mjx-c class="mjx-c29"></mjx-c></mjx-mo><mjx-mo class="mjx-n"><mjx-c class="mjx-c5D"></mjx-c></mjx-mo><mjx-mspace></mjx-mspace><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c79"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-texatom size="s" texclass="ORD"><mjx-mi class="mjx-i"><mjx-c class="mjx-c64"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c69"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c73"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c6F"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c65"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c64"></mjx-c></mjx-mi></mjx-texatom></mjx-script></mjx-msub><mjx-mo class="mjx-n" space="4"><mjx-c class="mjx-c3D"></mjx-c></mjx-mo><mjx-mi class="mjx-i" space="4"><mjx-c class="mjx-c79"></mjx-c></mjx-mi><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c5B"></mjx-c></mjx-mo><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c70"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c31"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-mo class="mjx-n"><mjx-c class="mjx-c28"></mjx-c></mjx-mo><mjx-msup><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-script style="vertical-align: 0.413em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msup><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-mn class="mjx-n" space="3"><mjx-c class="mjx-c32"></mjx-c></mjx-mn><mjx-msup><mjx-mi class="mjx-i"><mjx-c class="mjx-c79"></mjx-c></mjx-mi><mjx-script style="vertical-align: 0.413em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msup><mjx-mo class="mjx-n"><mjx-c class="mjx-c29"></mjx-c></mjx-mo><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-mn class="mjx-n" space="3"><mjx-c class="mjx-c32"></mjx-c></mjx-mn><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c70"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-mi class="mjx-i"><mjx-c class="mjx-c78"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c79"></mjx-c></mjx-mi><mjx-mo class="mjx-n"><mjx-c class="mjx-c5D"></mjx-c></mjx-mo></mjx-math><mjx-assistive-mml role="presentation" unselectable="on" display="block"><math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><msub><mi>x</mi><mrow><mi>d</mi><mi>i</mi><mi>s</mi><mi>t</mi><mi>o</mi><mi>r</mi><mi>t</mi><mi>e</mi><mi>d</mi></mrow></msub><mo>=</mo><mi>x</mi><mo>+</mo><mo stretchy="false">[</mo><mn>2</mn><msub><mi>p</mi><mn>1</mn></msub><mi>x</mi><mi>y</mi><mo>+</mo><msub><mi>p</mi><mn>2</mn></msub><mo stretchy="false">(</mo><msup><mi>r</mi><mn>2</mn></msup><mo>+</mo><mn>2</mn><msup><mi>x</mi><mn>2</mn></msup><mo stretchy="false">)</mo><mo stretchy="false">]</mo><mspace linebreak="newline"></mspace><msub><mi>y</mi><mrow><mi>d</mi><mi>i</mi><mi>s</mi><mi>t</mi><mi>o</mi><mi>r</mi><mi>t</mi><mi>e</mi><mi>d</mi></mrow></msub><mo>=</mo><mi>y</mi><mo>+</mo><mo stretchy="false">[</mo><msub><mi>p</mi><mn>1</mn></msub><mo stretchy="false">(</mo><msup><mi>r</mi><mn>2</mn></msup><mo>+</mo><mn>2</mn><msup><mi>y</mi><mn>2</mn></msup><mo stretchy="false">)</mo><mo>+</mo><mn>2</mn><msub><mi>p</mi><mn>2</mn></msub><mi>x</mi><mi>y</mi><mo stretchy="false">]</mo></math></mjx-assistive-mml></mjx-container>
</p>
<p>In short, we need to find five parameters, known as distortion coefficients given by:</p>
<p class="formulaDsp">
<mjx-container class="MathJax CtxtMenu_Attached_0" jax="CHTML" display="true" role="presentation" tabindex="0" ctxtmenu_counter="2" style="font-size: 118.2%; position: relative;"><mjx-math display="true" class="MJX-TEX" aria-hidden="true"><mjx-mi class="mjx-i"><mjx-c class="mjx-c44"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c69"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c73"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c6F"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c69"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c6F"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c6E"></mjx-c></mjx-mi><mjx-mstyle><mjx-mspace style="width: 0.278em;"></mjx-mspace></mjx-mstyle><mjx-mi class="mjx-i"><mjx-c class="mjx-c63"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c6F"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c65"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c66"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c66"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c69"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c63"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c69"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c65"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c6E"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c73"></mjx-c></mjx-mi><mjx-mo class="mjx-n" space="4"><mjx-c class="mjx-c3D"></mjx-c></mjx-mo><mjx-mo class="mjx-n" space="4"><mjx-c class="mjx-c28"></mjx-c></mjx-mo><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c6B"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c31"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-mspace style="width: 1em;"></mjx-mspace><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c6B"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-mspace style="width: 1em;"></mjx-mspace><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c70"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c31"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-mspace style="width: 1em;"></mjx-mspace><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c70"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c32"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-mspace style="width: 1em;"></mjx-mspace><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c6B"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mn class="mjx-n" size="s"><mjx-c class="mjx-c33"></mjx-c></mjx-mn></mjx-script></mjx-msub><mjx-mo class="mjx-n"><mjx-c class="mjx-c29"></mjx-c></mjx-mo></mjx-math><mjx-assistive-mml role="presentation" unselectable="on" display="block"><math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><mi>D</mi><mi>i</mi><mi>s</mi><mi>t</mi><mi>o</mi><mi>r</mi><mi>t</mi><mi>i</mi><mi>o</mi><mi>n</mi><mstyle scriptlevel="0"><mspace width="thickmathspace"></mspace></mstyle><mi>c</mi><mi>o</mi><mi>e</mi><mi>f</mi><mi>f</mi><mi>i</mi><mi>c</mi><mi>i</mi><mi>e</mi><mi>n</mi><mi>t</mi><mi>s</mi><mo>=</mo><mo stretchy="false">(</mo><msub><mi>k</mi><mn>1</mn></msub><mspace width="10pt"></mspace><msub><mi>k</mi><mn>2</mn></msub><mspace width="10pt"></mspace><msub><mi>p</mi><mn>1</mn></msub><mspace width="10pt"></mspace><msub><mi>p</mi><mn>2</mn></msub><mspace width="10pt"></mspace><msub><mi>k</mi><mn>3</mn></msub><mo stretchy="false">)</mo></math></mjx-assistive-mml></mjx-container>
</p>
<p>In addition to this, we need to some other information, like the intrinsic and extrinsic parameters of the camera. Intrinsic parameters are specific to a camera. They include information like focal length ( <mjx-container class="MathJax CtxtMenu_Attached_0" jax="CHTML" role="presentation" tabindex="0" ctxtmenu_counter="3" style="font-size: 118.2%; position: relative;"><mjx-math class="MJX-TEX" aria-hidden="true"><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c66"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mi class="mjx-i" size="s"><mjx-c class="mjx-c78"></mjx-c></mjx-mi></mjx-script></mjx-msub><mjx-mo class="mjx-n"><mjx-c class="mjx-c2C"></mjx-c></mjx-mo><mjx-msub space="2"><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c66"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mi class="mjx-i" size="s"><mjx-c class="mjx-c79"></mjx-c></mjx-mi></mjx-script></mjx-msub></mjx-math><mjx-assistive-mml role="presentation" unselectable="on" display="inline"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>f</mi><mi>x</mi></msub><mo>,</mo><msub><mi>f</mi><mi>y</mi></msub></math></mjx-assistive-mml></mjx-container>) and optical centers ( <mjx-container class="MathJax CtxtMenu_Attached_0" jax="CHTML" role="presentation" tabindex="0" ctxtmenu_counter="4" style="font-size: 118.2%; position: relative;"><mjx-math class="MJX-TEX" aria-hidden="true"><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c63"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mi class="mjx-i" size="s"><mjx-c class="mjx-c78"></mjx-c></mjx-mi></mjx-script></mjx-msub><mjx-mo class="mjx-n"><mjx-c class="mjx-c2C"></mjx-c></mjx-mo><mjx-msub space="2"><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c63"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mi class="mjx-i" size="s"><mjx-c class="mjx-c79"></mjx-c></mjx-mi></mjx-script></mjx-msub></mjx-math><mjx-assistive-mml role="presentation" unselectable="on" display="inline"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>c</mi><mi>x</mi></msub><mo>,</mo><msub><mi>c</mi><mi>y</mi></msub></math></mjx-assistive-mml></mjx-container>). The focal length and optical centers can be used to create a camera matrix, which can be used to remove distortion due to the lenses of a specific camera. The camera matrix is unique to a specific camera, so once calculated, it can be reused on other images taken by the same camera. It is expressed as a 3x3 matrix:</p>
<p class="formulaDsp">
<mjx-container class="MathJax CtxtMenu_Attached_0" jax="CHTML" display="true" role="presentation" tabindex="0" ctxtmenu_counter="5" style="font-size: 118.2%; position: relative;"><mjx-math display="true" class="MJX-TEX" aria-hidden="true"><mjx-mi class="mjx-i"><mjx-c class="mjx-c63"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c61"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c6D"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c65"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c61"></mjx-c></mjx-mi><mjx-mstyle><mjx-mspace style="width: 0.278em;"></mjx-mspace></mjx-mstyle><mjx-mi class="mjx-i"><mjx-c class="mjx-c6D"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c61"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c74"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c72"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c69"></mjx-c></mjx-mi><mjx-mi class="mjx-i"><mjx-c class="mjx-c78"></mjx-c></mjx-mi><mjx-mo class="mjx-n" space="4"><mjx-c class="mjx-c3D"></mjx-c></mjx-mo><mjx-mrow space="4"><mjx-mo class="mjx-n"><mjx-stretchy-v class="mjx-c5B" style="height: 3.845em; vertical-align: -1.672em;"><mjx-beg><mjx-c></mjx-c></mjx-beg><mjx-ext><mjx-c></mjx-c></mjx-ext><mjx-end><mjx-c></mjx-c></mjx-end><mjx-mark></mjx-mark></mjx-stretchy-v></mjx-mo><mjx-mtable style="min-width: 4.718em;"><mjx-table><mjx-itable><mjx-mtr><mjx-mtd style="padding-right: 0.5em; padding-bottom: 0.2em;"><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c66"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mi class="mjx-i" size="s"><mjx-c class="mjx-c78"></mjx-c></mjx-mi></mjx-script></mjx-msub><mjx-tstrut></mjx-tstrut></mjx-mtd><mjx-mtd style="padding-left: 0.5em; padding-right: 0.5em; padding-bottom: 0.2em;"><mjx-mn class="mjx-n"><mjx-c class="mjx-c30"></mjx-c></mjx-mn><mjx-tstrut></mjx-tstrut></mjx-mtd><mjx-mtd style="padding-left: 0.5em; padding-bottom: 0.2em;"><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c63"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mi class="mjx-i" size="s"><mjx-c class="mjx-c78"></mjx-c></mjx-mi></mjx-script></mjx-msub><mjx-tstrut></mjx-tstrut></mjx-mtd></mjx-mtr><mjx-mtr><mjx-mtd style="padding-right: 0.5em; padding-top: 0.2em; padding-bottom: 0.2em;"><mjx-mn class="mjx-n"><mjx-c class="mjx-c30"></mjx-c></mjx-mn><mjx-tstrut></mjx-tstrut></mjx-mtd><mjx-mtd style="padding: 0.2em 0.5em;"><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c66"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mi class="mjx-i" size="s"><mjx-c class="mjx-c79"></mjx-c></mjx-mi></mjx-script></mjx-msub><mjx-tstrut></mjx-tstrut></mjx-mtd><mjx-mtd style="padding-left: 0.5em; padding-top: 0.2em; padding-bottom: 0.2em;"><mjx-msub><mjx-mi class="mjx-i" noic="true"><mjx-c class="mjx-c63"></mjx-c></mjx-mi><mjx-script style="vertical-align: -0.15em;"><mjx-mi class="mjx-i" size="s"><mjx-c class="mjx-c79"></mjx-c></mjx-mi></mjx-script></mjx-msub><mjx-tstrut></mjx-tstrut></mjx-mtd></mjx-mtr><mjx-mtr><mjx-mtd style="padding-right: 0.5em; padding-top: 0.2em;"><mjx-mn class="mjx-n"><mjx-c class="mjx-c30"></mjx-c></mjx-mn><mjx-tstrut></mjx-tstrut></mjx-mtd><mjx-mtd style="padding-left: 0.5em; padding-right: 0.5em; padding-top: 0.2em;"><mjx-mn class="mjx-n"><mjx-c class="mjx-c30"></mjx-c></mjx-mn><mjx-tstrut></mjx-tstrut></mjx-mtd><mjx-mtd style="padding-left: 0.5em; padding-top: 0.2em;"><mjx-mn class="mjx-n"><mjx-c class="mjx-c31"></mjx-c></mjx-mn><mjx-tstrut></mjx-tstrut></mjx-mtd></mjx-mtr></mjx-itable></mjx-table></mjx-mtable><mjx-mo class="mjx-n"><mjx-stretchy-v class="mjx-c5D" style="height: 3.845em; vertical-align: -1.672em;"><mjx-beg><mjx-c></mjx-c></mjx-beg><mjx-ext><mjx-c></mjx-c></mjx-ext><mjx-end><mjx-c></mjx-c></mjx-end><mjx-mark></mjx-mark></mjx-stretchy-v></mjx-mo></mjx-mrow></mjx-math><mjx-assistive-mml role="presentation" unselectable="on" display="block"><math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><mi>c</mi><mi>a</mi><mi>m</mi><mi>e</mi><mi>r</mi><mi>a</mi><mstyle scriptlevel="0"><mspace width="thickmathspace"></mspace></mstyle><mi>m</mi><mi>a</mi><mi>t</mi><mi>r</mi><mi>i</mi><mi>x</mi><mo>=</mo><mrow data-mjx-texclass="INNER"><mo data-mjx-texclass="OPEN">[</mo><mtable columnspacing="1em" rowspacing="4pt"><mtr><mtd><msub><mi>f</mi><mi>x</mi></msub></mtd><mtd><mn>0</mn></mtd><mtd><msub><mi>c</mi><mi>x</mi></msub></mtd></mtr><mtr><mtd><mn>0</mn></mtd><mtd><msub><mi>f</mi><mi>y</mi></msub></mtd><mtd><msub><mi>c</mi><mi>y</mi></msub></mtd></mtr><mtr><mtd><mn>0</mn></mtd><mtd><mn>0</mn></mtd><mtd><mn>1</mn></mtd></mtr></mtable><mo data-mjx-texclass="CLOSE">]</mo></mrow></math></mjx-assistive-mml></mjx-container>
</p>
<p>Extrinsic parameters corresponds to rotation and translation vectors which translates a coordinates of a 3D point to a coordinate system.</p>
<p>For stereo applications, these distortions need to be corrected first. To find these parameters, we must provide some sample images of a well defined pattern (e.g. a chess board). We find some specific points of which we already know the relative positions (e.g. square corners in the chess board). We know the coordinates of these points in real world space and we know the coordinates in the image, so we can solve for the distortion coefficients. For better results, we need at least 10 test patterns.</p>
<h1><a class="anchor" id="autotoc_md1159"></a>
Code</h1>
<p>As mentioned above, we need at least 10 test patterns for camera calibration. OpenCV comes with some images of a chess board (see samples/data/left01.jpg – left14.jpg), so we will utilize these. Consider an image of a chess board. The important input data needed for calibration of the camera is the set of 3D real world points and the corresponding 2D coordinates of these points in the image. 2D image points are OK which we can easily find from the image. (These image points are locations where two black squares touch each other in chess boards)</p>
<p>What about the 3D points from real world space? Those images are taken from a static camera and chess boards are placed at different locations and orientations. So we need to know <mjx-container class="MathJax CtxtMenu_Attached_0" jax="CHTML" role="presentation" tabindex="0" ctxtmenu_counter="6" style="font-size: 118.2%; position: relative;"><mjx-math class="MJX-TEX" aria-hidden="true"><mjx-mo class="mjx-n"><mjx-c class="mjx-c28"></mjx-c></mjx-mo><mjx-mi class="mjx-i"><mjx-c class="mjx-c58"></mjx-c></mjx-mi><mjx-mo class="mjx-n"><mjx-c class="mjx-c2C"></mjx-c></mjx-mo><mjx-mi class="mjx-i" space="2"><mjx-c class="mjx-c59"></mjx-c></mjx-mi><mjx-mo class="mjx-n"><mjx-c class="mjx-c2C"></mjx-c></mjx-mo><mjx-mi class="mjx-i" space="2"><mjx-c class="mjx-c5A"></mjx-c></mjx-mi><mjx-mo class="mjx-n"><mjx-c class="mjx-c29"></mjx-c></mjx-mo></mjx-math><mjx-assistive-mml role="presentation" unselectable="on" display="inline"><math xmlns="http://www.w3.org/1998/Math/MathML"><mo stretchy="false">(</mo><mi>X</mi><mo>,</mo><mi>Y</mi><mo>,</mo><mi>Z</mi><mo stretchy="false">)</mo></math></mjx-assistive-mml></mjx-container> values. But for simplicity, we can say chess board was kept stationary at XY plane, (so Z=0 always) and camera was moved accordingly. This consideration helps us to find only X,Y values. Now for X,Y values, we can simply pass the points as (0,0), (1,0), (2,0), ... which denotes the location of points. In this case, the results we get will be in the scale of size of chess board square. But if we know the square size, (say 30 mm), we can pass the values as (0,0), (30,0), (60,0), ... . Thus, we get the results in mm. (In this case, we don't know square size since we didn't take those images, so we pass in terms of square size).</p>
<p>3D points are called <b>object points</b> and 2D image points are called <b>image points.</b></p>
<h2><a class="anchor" id="autotoc_md1160"></a>
Setup</h2>
<p>So to find pattern in chess board, we can use the function, <b><a class="el keychainify-checked" href="https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga93efa9b0aa890de240ca32b11253dd4a" title="Finds the positions of internal corners of the chessboard.">cv.findChessboardCorners()</a></b>. We also need to pass what kind of pattern we are looking for, like 8x8 grid, 5x5 grid etc. In this example, we use 7x6 grid. (Normally a chess board has 8x8 squares and 7x7 internal corners). It returns the corner points and retval which will be True if pattern is obtained. These corners will be placed in an order (from left-to-right, top-to-bottom)</p>
<dl class="section note"><dt>Note</dt><dd>This function may not be able to find the required pattern in all the images. So, one good option is to write the code such that, it starts the camera and check each frame for required pattern. Once the pattern is obtained, find the corners and store it in a list. Also, provide some interval before reading next frame so that we can adjust our chess board in different direction. Continue this process until the required number of good patterns are obtained. Even in the example provided here, we are not sure how many images out of the 14 given are good. Thus, we must read all the images and take only the good ones.</dd>
<dd>
Instead of chess board, we can alternatively use a circular grid. In this case, we must use the function <b><a class="el keychainify-checked" href="https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga7f02cd21c8352142890190227628fa80" title="Finds centers in the grid of circles.">cv.findCirclesGrid()</a></b> to find the pattern. Fewer images are sufficient to perform camera calibration using a circular grid.</dd></dl>
<p>Once we find the corners, we can increase their accuracy using <b><a class="el keychainify-checked" href="https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#ga354e0d7c86d0d9da75de9b9701a9a87e" title="Refines the corner locations.">cv.cornerSubPix()</a></b>. We can also draw the pattern using <b><a class="el keychainify-checked" href="https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga6a10b0bb120c4907e5eabbcd22319022" title="Renders the detected chessboard corners.">cv.drawChessboardCorners()</a></b>. All these steps are included in below code:</p>





In [ ]:
import glob
 
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('assets/left*.jpg')
print(images)
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6), None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,6), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)
 
cv2.destroyAllWindows()

<h2><a class="anchor" id="autotoc_md1161"></a>
Calibration</h2>
<p>Now that we have our object points and image points, we are ready to go for calibration. We can use the function, <b><a class="el keychainify-checked" href="https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d" title="Finds the camera intrinsic and extrinsic parameters from several views of a calibration pattern.">cv.calibrateCamera()</a></b> which returns the camera matrix, distortion coefficients, rotation and translation vectors etc. </p>

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

<h2><a class="anchor" id="autotoc_md1162"></a>
Undistortion</h2>
<p>Now, we can take an image and undistort it. OpenCV comes with two methods for doing this. However first, we can refine the camera matrix based on a free scaling parameter using <b><a class="el keychainify-checked" href="https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga7a6c4e032c97f03ba747966e6ad862b1" title="Returns the new camera intrinsic matrix based on the free scaling parameter.">cv.getOptimalNewCameraMatrix()</a></b>. If the scaling parameter alpha=0, it returns undistorted image with minimum unwanted pixels. So it may even remove some pixels at image corners. If alpha=1, all pixels are retained with some extra black images. This function also returns an image ROI which can be used to crop the result.</p>
<p>So, we take a new image (left12.jpg in this case. That is the first image in this chapter) </p>


In [ ]:
img = cv2.imread('assets/left12.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

<h3><a class="anchor" id="autotoc_md1163"></a>
Using cv.undistort()</h3>
<p>This is the easiest way. Just call the function and use ROI obtained above to crop the result. </p>

In [ ]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png', dst)

<p> Still, both the methods give the same result. See the result below:</p>
<div class="image">
<img src="https://docs.opencv.org/4.x/calib_result.jpg" alt="">
<div class="caption">
image</div></div>
<p>You can see in the result that all the edges are straight.</p>
<p>Now you can store the camera matrix and distortion coefficients using write functions in NumPy (np.savez, np.savetxt etc) for future uses.</p>
<h1><a class="anchor" id="autotoc_md1165"></a>
Re-projection Error</h1>
<p>Re-projection error gives a good estimation of just how exact the found parameters are. The closer the re-projection error is to zero, the more accurate the parameters we found are. Given the intrinsic, distortion, rotation and translation matrices, we must first transform the object point to image point using <b><a class="el keychainify-checked" href="https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga1019495a2c8d1743ed5cc23fa0daff8c" title="Projects 3D points to an image plane.">cv.projectPoints()</a></b>. Then, we can calculate the absolute norm between what we got with our transformation and the corner finding algorithm. To find the average error, we calculate the arithmetical mean of the errors calculated for all the calibration images. </p>

In [ ]:
mean_error = 0
for i in range(len(objpoints)):
 imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
 error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
 mean_error += error
 
print( "total error: {}".format(mean_error/len(objpoints)) )

## 4. Depth Map from Stereo Images
This tutorial was originally from [here](https://docs.opencv.org/4.x/dd/d53/tutorial_py_depthmap.html)

Below code snippet shows a simple procedure to create a disparity map.



In [ ]:
imgL = cv2.imread('assets/tsukuba_l.png', cv2.IMREAD_GRAYSCALE)
imgR = cv2.imread('assets/tsukuba_r.png', cv2.IMREAD_GRAYSCALE)
 
stereo = cv2.StereoBM.create(numDisparities=16, blockSize=15)
disparity = stereo.compute(imgL,imgR)
plt.imshow(disparity,'gray')
plt.show()

There are some parameters when you get familiar with StereoBM, and you may need to fine tune the parameters to get better and smooth results. Parameters:

- texture_threshold: filters out areas that don't have enough texture for reliable matching
- Speckle range and size: Block-based matchers often produce "speckles" near the boundaries of objects, where the matching window catches the foreground on one side and the background on the other. In this scene it appears that the matcher is also finding small spurious matches in the projected texture on the table. To get rid of these artifacts we post-process the disparity image with a speckle filter controlled by the speckle_size and speckle_range parameters. speckle_size is the number of pixels below which a disparity blob is dismissed as "speckle." speckle_range controls how close in value disparities must be to be considered part of the same blob.
- Number of disparities: How many pixels to slide the window over. The larger it is, the larger the range of visible depths, but more computation is required.
- min_disparity: the offset from the x-position of the left pixel at which to begin searching.
- uniqueness_ratio: Another post-filtering step. If the best matching disparity is not sufficiently better than every other disparity in the search range, the pixel is filtered out. You can try tweaking this if texture_threshold and the speckle filtering are still letting through spurious matches.
- prefilter_size and prefilter_cap: The pre-filtering phase, which normalizes image brightness and enhances texture in preparation for block matching. Normally you should not need to adjust these.
These parameters are set with dedicated setters and getters after the algoritm initialization, such as setTextureThreshold, setSpeckleRange, setUniquenessRatio, and more. See cv::StereoBM documentation for details.

These parameters are set with dedicated setters and getters after the algoritm initialization, such as setTextureThreshold, setSpeckleRange, setUniquenessRatio, and more. See cv::StereoBM documentation for details.



In [ ]:
imgL = cv2.imread('assets/plantL.jpeg', cv2.IMREAD_GRAYSCALE)
imgR = cv2.imread('assets/plantR.jpeg', cv2.IMREAD_GRAYSCALE)
plt.imshow(imgR,'gray')
plt.show()


stereo = cv2.StereoBM.create(numDisparities=128, blockSize=15)
disparity = stereo.compute(imgL,imgR)
plt.imshow(disparity,'gray')
plt.show()